In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.base import TransformerMixin
from sklearn.feature_selection import SelectKBest
import itertools
from sklearn.linear_model import SGDRegressor
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline


import pandas
import numpy as np
from sklearn.grid_search import GridSearchCV
dfNotNull = pandas.read_csv('Long.csv')
dfNotNull = dfNotNull[dfNotNull.noCPT == 0]
dfNotNull.head()
print dfNotNull.Procedure3.value_counts()

In [82]:
print "get null scores"

numFolds = 5
y = dfNotNull['TotalTimeMin']
X = dfNotNull['Procedure4']
abse =[]
sqrd =[]

import CleanDefs
for i in range(1,25):
    kf = KFold(len(X), numFolds, shuffle=True)
    for train_indices, test_indices in kf:
        features_train = X[train_indices]
        response_train= y[train_indices]
        features_test = X[test_indices]
        response_test = y[test_indices]

        dfT = pandas.DataFrame()
        dfT['Procedure4'] = features_train
        dfT['TotalTimeMin']= response_train
        dfT['AverageProcTime']=dfT['TotalTimeMin'].mean()
        
        dfR = pandas.DataFrame()
        dfR['Procedure4'] = features_test
        dfR['TotalTimeMin']= response_test
        dfR['AverageProcTime']=dfT['TotalTimeMin'].mean()

        y1 = dfR['TotalTimeMin']
        X1 = dfR['AverageProcTime']
        differences = (y1 - X1).abs()
        abse += [differences.mean()]
        sqrd += [(differences**2).mean()]
        
        
print "Total Average Null:"
print np.array(abse).mean()
print np.sqrt(np.array(sqrd)).mean()


numFolds = 5

y = dfNotNull['TotalTimeMin']
X = dfNotNull['Procedure4']

abse =[]
sqrd =[]
for i in range(1,25):
    kf = KFold(len(X), numFolds, shuffle=True)
    smlSqrd =[]
    for train_indices, test_indices in kf:
        features_train = X[train_indices]
        response_train= y[train_indices]
        features_test = X[test_indices]
        response_test = y[test_indices]

        dfT = pandas.DataFrame()
        dfT['Procedure4'] = features_train
        dfT['TotalTimeMin']= response_train
        dfT['AverageProcTime']=dfT['Procedure4'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )
        dfR = pandas.DataFrame()
        dfR['Procedure4'] = features_test
        dfR['TotalTimeMin']= response_test
        dfR['AverageProcTime']=dfR['Procedure4'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )
        
        y1 = dfR['TotalTimeMin']
        X1 = dfR['AverageProcTime']
        differences = (y1 - X1).abs()
        abse += [differences.mean()]
        
        smlSqrd += [(differences**2).mean()]
    #print np.array([smlSqrd])
    sqrd += [np.sqrt(np.array(smlSqrd)).mean()]
print "Procedure Average Null:"
print np.array(abse).mean()

print np.array(sqrd).mean()




get null scores
Total Average Null:
11.1198876234
17.3361957946
Procedure Average Null:
6.60038183139
10.2319012877


In [215]:
print "Different Features sets and corr plots"

Procedure3feat = ['Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Facet Joint Injection', 'Cervical/Thoracic ESI'] 
Procedure3feat += ['Cervical/Thoracic Medial Branch Block', 'Spinal Cord Stimulator', 'Nerve Block', 'Lumbar Medial Branch Block']
Procedure3feat += ['Lumbar ESI', 'Joint Injection, other']
                  
Procedure4feat = ['Radiofrequency','ESI', 'Medial Branch Block', 'Injection',  'Nerve Block 2','Spinal Cord Stimulator 2']
                  
ClassFeat = ['3', 'Gender', '1r']

NonScaledContfeat  = ['Age','BMI','HealthCodesLen', 'SurgicalHistoryLen']

ScaledContfeat  = ['AgeScaled','BMIScaled','HealthCodesLenScaled', 'SurgicalHistoryLenScaled']

HealthCodes = ['M54', 'G25', 'M51', 'M50', 'M53', 'G54', 'E78', 'C177', 'D51', 'G3', 'G2', 'N28', 'I73', 'M48', 'M43', 'M41', 'M46', 'M47', 'Z79', 'F32', 'G57', 'D68', 'Q76', 'Z72', 'Y92', 'T85', 'I69', 'C182', 'M32', 'G40','F03', 'G47', 'C178', 'Y83', 'Y84', 'I50', 'C45', 'E05', 'M25', 'F12', 'F17', 'K74', 'I48', 'I42', 'C79', 'I25','E11', 'J45', 'C6', 'L40', 'E21', 'C189', 'N1', 'M96', 'M10', 'M12', 'M19', 'F41', 'Z01', 'J44', 'M79', 'J43', 'Z80','I89', 'Z85', 'Z87', 'Z86', 'Z89', 'Z88', 'C95', 'I1', 'N40', 'M84', 'M81', 'M06', 'G95', 'G90', 'R53', 'R56', 'E06', 'E07', 'K21', 'E03', 'Z96', 'Z94', 'Z95', 'Z92', 'Z90', 'Z91', 'Z98', 'E89', 'C81', 'G82', 'R26', 'Q61']

HealthCodes2 = ['C', 'E', 'D', 'G', 'F', 'I', 'K', 'J', 'M', 'L', 'N', 'Q', 'R', 'T', 'Y', 'Z']

CPTcodes = ['20610', '64479', '64640', '63650', '64483', '64480', '64634', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64494', '64495', '64633', '64490', '64491', '64492', '64493', '64450', '27096']

InsuranceCodes = ['BLUE CROSS','BLUE SHIELD','MEDICARE']
print dfNotNull.TotalTimeMin.mean()
sigCodesList = []
for i in HealthCodes:
    if len(dfNotNull[dfNotNull[i] == 1]) > 20 and abs(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean() - 34.18) >=3 :
        print i + " = " + str(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean()) + ", " + str(len(dfNotNull[dfNotNull[i] == 1]))
        sigCodesList += [i]
        

for i in HealthCodes2:
    if len(dfNotNull[dfNotNull[i] == 1]) > 20 and abs(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean() - 34.18) >=0 :
        print i + " = " + str(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean()) + ", " + str(len(dfNotNull[dfNotNull[i] == 1]))
        sigCodesList += [i]
cptCodesList = []
for i in CPTcodes:
    if len(dfNotNull[dfNotNull[i] == 1]) > 0 and abs(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean() - 34.18) >=0 :
        print i + " = " + str(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean()) + ", " + str(len(dfNotNull[dfNotNull[i] == 1]))
        cptCodesList += [i]
    
    
feature_cols = Procedure3feat + Procedure4feat + ClassFeat + NonScaledContfeat + sigCodesList
corrDF = dfNotNull[ NonScaledContfeat + ['TotalTimeMin']]
corrDF.corr()
print sigCodesList

Different Features sets and corr plots
34.1856725146
F32 = 42.347826087, 23
F17 = 31.0612244898, 98
M96 = 43.0961538462, 104
Z85 = 40.0833333333, 36
Z87 = 37.2727272727, 22
Z95 = 37.5675675676, 37
E = 33.864628821, 229
G = 40.1857142857, 70
F = 33.496, 125
I = 34.5345454545, 275
J = 35.4285714286, 84
M = 34.1785714286, 672
Z = 34.4704225352, 355
20610 = 27.6666666667, 3
64479 = 30.0, 1
64640 = 50.0, 1
63650 = 93.32, 25
64483 = 23.9052631579, 190
64480 = 30.0, 1
64634 = 39.5961538462, 52
64635 = 41.7794871795, 195
64636 = 41.7794871795, 195
64484 = 23.90625, 160
20605 = 29.0, 1
63685 = 113.0, 16
63688 = 89.0, 1
62311 = 28.0909090909, 11
62310 = 25.0952380952, 42
63661 = 89.0, 1
64494 = 27.8405797101, 138
64495 = 28.4942528736, 87
64633 = 39.5961538462, 52
64490 = 30.2203389831, 59
64491 = 30.2857142857, 56
64492 = 29.3076923077, 39
64493 = 27.8967741935, 155
64450 = 33.0, 1
27096 = 18.8, 5
['F32', 'F17', 'M96', 'Z85', 'Z87', 'Z95', 'E', 'G', 'F', 'I', 'J', 'M', 'Z']


In [10]:
print """
Run Linear Regression Train-Test Split ****
"""
feature_cols += ['Spinal Cord Stimulator']
X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']


abse = []
sqrd = []
for i in range(1,2):
    features_train, features_test, response_train, response_test \
    = train_test_split(X, y, random_state=4)
    # instantiate and fit

    
    linreg = LinearRegression()
    linreg.fit(features_train, response_train)
    print type(features_test)
    predictionDF = pandas.DataFrame()
    predictionDF['Predictions'] = linreg.predict(features_test)
    predictionDF['Actual'] = response_test.values
    differences = (predictionDF['Predictions'] - predictionDF['Actual']).abs()
    abse += [differences.mean()]
    sqrd += [(differences**2).mean()]
print np.array(abse).mean()
print np.sqrt(np.array(sqrd)).mean() 
f = lambda x:int(round(x/15.)*15)


print predictionDF.head(15)
print zip(feature_cols,linreg.coef_) 



Run Linear Regression Train-Test Split ****

<class 'pandas.core.frame.DataFrame'>
6.09559734025
9.29382480413
    Predictions  Actual
0     19.872998      18
1     29.632889      26
2     23.859569      23
3     19.872998      27
4     25.646317      27
5     31.039443      44
6     37.028868      31
7    101.943957     102
8     42.978185      49
9     25.646317      20
10    42.978185      36
11    40.770401      42
12    31.039443      25
13    42.978185      51
14    28.831659      37
[('Lumbar Radiofrequency', 5.9493174695004312), ('3', 1.7787869822836804), ('Radiofrequency', -51.971131993865313), ('ESI', -65.140430880427076), ('Medial Branch Block', -57.960556635538822), ('Injection', -59.367111053360595), ('Nerve Block 2', -61.977989848280245), ('1r', -2.2077846265808923), ('Spinal Cord Stimulator', 12.943957223988743)]


AttributeError: 'DataFrame' object has no attribute 'cptCodes'

In [329]:

        
        
        
print """
Run Linear Regression w/ Iteartive Feature Selection ****
"""
feature_cols = ['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64494', '64495', '64633', '64490', '64491', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
feature_cols = ['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r', 'Gender', 'F17', 'Age']
feature_cols += ['64640', '63650', '64634', '64480', '64635']
X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']



def cross_val_Lin_ITER(X, y):
    
    
    linreg2 = LinearRegression()
   
    
    mean2escores3 = cross_val_score(linreg2 , X, y, cv=5, scoring='mean_squared_error')
    meanAbsScores3= cross_val_score(linreg2 , X, y, cv=5, scoring='mean_absolute_error')
    return (mean2escores3,meanAbsScores3) # return average R2

mean2escorelist = []
for L in range(27, len(feature_cols)+1):
    print L
    print "good score list"
    print mean2escorelist.sort()
    
   
    for subset in itertools.combinations(feature_cols, L):
        
        Subset = list(subset)
        X = dfNotNull[Subset]
        cvScores= cross_val_Lin_ITER(X, y)
        
        mean2escores = cvScores[0]
        mean2escores = np.mean(np.sqrt(-mean2escores))
        meanAbsScores= cvScores[1]
        meanAbsScores= -meanAbsScores.mean()
        
        if mean2escores < 8.89 and meanAbsScores <  6.10:
            mean2escorelist+= [mean2escores]
            
            print subset    
            print "root mean squared error"
            
            print mean2escores
            print "mean Abs Scores"

            print meanAbsScores
            print '\n'
            



Run Linear Regression w/ Iteartive Feature Selection ****



In [ ]:
print 5

In [4]:
print "Standar Linear Regression W/ Cross Validation"

cptCodesList = []

feature_cols = ['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', 
                '62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17',  '1r', 
                'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
len(feature_cols)
X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']
print len(X)
print len(y)
    
    
linreg4 = LinearRegression()


mean2escores3 = cross_val_score(linreg4 , X, y, cv=5, scoring='mean_squared_error')
mean2escores4 = cross_val_score(linreg4 , X, y, cv=5, scoring='mean_absolute_error')

print mean2escores3

mean2escores =np.sqrt(-mean2escores3).mean()
meanAbsScores= -mean2escores4.mean()


print feature_cols
print "mean squared error"           
print mean2escores
print "mean Abs Scores"

print meanAbsScores
print '\n'

print len(feature_cols2)

Standar Linear Regression W/ Cross Validation
672
672
[-141.8490452   -93.35778642  -61.82801688  -51.32990826  -55.10773072]
['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
mean squared error
8.80464997546
mean Abs Scores
6.09908443762




NameError: name 'feature_cols2' is not defined

In [7]:
print "Standard SDGC W/ GridSearch"
feature_cols =['64640', '64484', '20605', '63685', '63688', '62311', 
                '62310', '63661',  '64450', '27096', 'Gender', 'Age', 'F17',  '1r', 
                'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']#feature_cols += ['Gender'] #+['F17'] + ['Age'] 
tf= dfNotNull[feature_cols]
X = tf
y = dfNotNull['TotalTimeMin']

sgd = SGDRegressor()



param_grid = {'loss':['squared_loss'], 
              'penalty':['none', 'l2', 'l1', 'elasticnet'],
              'alpha': [1e-4,1e-3,1e-2,1e-1,1,1e1],
              'n_iter': [10,100,1000]
             }

grid2= GridSearchCV(sgd , param_grid, cv=5, scoring='mean_absolute_error').fit(X,y)
grid1 = GridSearchCV(sgd, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
grid1.best_estimator_
print 555
mean2escores3 = cross_val_score(grid1.best_estimator_ , X, y, cv=5, scoring='mean_squared_error')
mean2escores4 = cross_val_score(grid2.best_estimator_ , X, y, cv=5, scoring='mean_absolute_error')



mean2escores =np.sqrt(-mean2escores3).mean()
meanAbsScores= mean2escores4
meanAbsScores= -meanAbsScores.mean()


print feature_cols   
print "mean squared error"
print mean2escores3.mean()           
print mean2escores
print "mean Abs Scores"

print meanAbsScores
print '\n'


Standard SDGC W/ GridSearch
555
['64640', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64450', '27096', 'Gender', 'Age', 'F17', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
mean squared error
-3102549109.73
46905.5320745
mean Abs Scores
15881.6881392




In [199]:
print dfNotNull.AgeScaled

0     -1.056677
1     -1.917798
2      1.167884
3      0.665564
4      1.598444
5      0.019724
6      2.100765
7      1.167884
8     -1.702518
9      1.167884
10    -0.339077
11     0.235004
12     1.024364
13    -1.056677
14     1.096124
15     0.665564
16    -0.123797
17    -1.271957
18     1.311404
19     0.306764
20     0.163244
21     0.593804
22    -0.554357
23     0.522044
24     2.459565
25    -0.195557
26    -1.558997
27     0.019724
28    -1.128437
29     0.450284
         ...   
654    0.091484
655    1.239644
656    1.526684
657    1.167884
658    0.378524
659   -1.487237
660    0.737324
661    0.091484
662   -0.913157
663    0.952604
664    1.167884
665   -2.204838
666    1.526684
667    0.737324
668    0.809084
669    0.665564
670    0.163244
671   -0.697877
672    0.450284
673   -1.056677
674   -1.487237
675    0.378524
676    0.378524
677    1.885485
678   -1.415477
679    0.306764
680   -0.913157
681    0.163244
682   -2.778918
683   -1.056677
Name: AgeScaled, dtype: 

In [9]:
print mean2escores

nan


In [200]:
print """
Run KNN Regression Cross Val All Feature Combos Piped and non piped Scalar ****
"""


"scaling pipe"
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union

from sklearn.neighbors import KNeighborsRegressor
"Scale values in the DF"   
from sklearn.base import TransformerMixin
#from sklearn import preprocessing

class FeatureScalarTransformer(TransformerMixin):
        def transform(self,X,  **transform_params):
            if 'BMI' in X:
                #X['BMI'] = preprocessing.scale(X['BMI'])
                scal = StandardScaler().fit(X['BMI'])
                X['BMI'] = scal.transform(X['BMI'])
            if 'Age' in X:
                #X['Age']= preprocessing.scale(X['Age'].astype('float'))
                scal = StandardScaler().fit(X['Age'])
                X['Age']=  scal.transform(X['Age'])
            if 'SurgicalHistoryLen' in X:
                #X['SurgicalHistoryLen'] = preprocessing.scale(X['SurgicalHistoryLen'].astype('float')
                scal = StandardScaler().fit(X['SurgicalHistoryLen'])                
                X['SurgicalHistoryLen'] = scal.transform(X['SurgicalHistoryLen'])
            if 'HealthCodesLen' in X:
                #X['HealthCodesLen'] = preprocessing.scale(X['HealthCodesLen'].astype('float'))
                scal = StandardScaler().fit(X['HealthCodesLen'])
                X['HealthCodesLen'] = scal.transform(X['HealthCodesLen'])
            
            
            return X
           

        def fit(self, X, y=None, **fit_params):
            return self



 


Run KNN Regression Cross Val All Feature Combos Piped and non piped Scalar ****



In [238]:
def feature_list_creator(tf_features):
    b = str(tf_features).replace('[', 'r').replace("  "," ").replace("\n","").split(" ")
    b =[x for x in b if x != 'r']
    feature_list = []
    for num,i in enumerate(b):
        if 'Tr' in i:
            feature_list += [feature_cols[num]]
        else:
            pass
    return feature_list

In [322]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
print "Run KNN regression without scalar and with cross validation xyy"


feature_cols = ['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', 
                '62310', '63661', '64494', '64495', '64633', '64490', '64491', '64492', '64493', '64450', '27096', 
                'Gender', 'Age', 'F17', '3', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']
for i in range (1, len(feature_cols)+1):
    #gridsearch with select kbest
    neighbs_k = range(1, 30)
    param_grid = dict(neigh__n_neighbors=neighbs_k )
    neigh = KNeighborsRegressor()
    filter1 = SelectKBest(k=i) # select the best 2 features
    pipe = Pipeline([('anova', filter1), ('neigh', neigh)])
    grid1 = GridSearchCV(pipe, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
    grid2= GridSearchCV(pipe, param_grid, cv=5, scoring='mean_absolute_error').fit(X,y)
    tf_features=str(grid1.best_estimator_.named_steps['anova'].get_support())
    abs_features=str(grid2.best_estimator_.named_steps['anova'].get_support())
    print grid1.best_estimator_
    mean2escores3 = cross_val_score(grid1.best_estimator_ , X, y, cv=5, scoring='mean_squared_error')
    mean2escores4 = cross_val_score(grid2.best_estimator_ , X, y, cv=5, scoring='mean_absolute_error')


    mean2escores =np.sqrt(-mean2escores3).mean()
    meanAbsScores= mean2escores4
    meanAbsScores= -meanAbsScores.mean()


    print feature_list_creator(tf_features)
    print "mean squared error"
    print mean2escores3.mean()           
    print mean2escores
    print "mean Abs Scores"

    print meanAbsScores
    print '\n'

    
    
   
    
   

Run KNN regression without scalar and with cross validation xyy
Pipeline(steps=[('anova', SelectKBest(k=1, score_func=<function f_classif at 0x000000000A934668>)), ('neigh', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=9, p=2,
          weights='uniform'))])
['63661']
mean squared error
-280.390606365
16.6271259022
mean Abs Scores
10.5679049198


Pipeline(steps=[('anova', SelectKBest(k=2, score_func=<function f_classif at 0x000000000A934668>)), ('neigh', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=12, p=2,
          weights='uniform'))])
['63688', '63661']
mean squared error
-279.634881457
16.5943045245
mean Abs Scores
10.545590105


Pipeline(steps=[('anova', SelectKBest(k=3, score_func=<function f_classif at 0x000000000A934668>)), ('neigh', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_

KeyboardInterrupt: 

In [244]:
print "Random Forest Grid Search***"

feature_cols =  Procedure4feat + Procedure3feat +ClassFeat + NonScaledContfeat + sigCodesList
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection','Medial Branch Block' ]#

feature_cols =['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64494', '64495', '64633', '64490', '64491', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']#

X= dfNotNull[feature_cols]
n_ests = range(1, 20, 2)
#n_ests = range(1, 90, 2)
max_feats = range(1,len(feature_cols)+1)
#max_feats = range(2,13)
param_grid = dict(n_estimators=n_ests,max_features =max_feats)


rfreg = RandomForestRegressor()

grid = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X,y)

print np.sqrt(-grid.best_score_)


Random Forest Grid Search***
10.3295627032


In [245]:
print "BEST FEATURES FOR RANDOM FOREST GRID SEARCH AND PREDICTIONS"
print grid.best_estimator_
print np.sqrt(-grid.best_score_)
print pandas.DataFrame({'feature':feature_cols, 'importance':grid.best_estimator_.feature_importances_}).sort('importance')
RFPredictions =grid.best_estimator_.predict(X)

BEST FEATURES FOR RANDOM FOREST GRID SEARCH AND PREDICTIONS
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=17, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=11, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
10.3295627032
                  feature  importance
7                   20605    0.000010
3                   64480    0.000033
20                  64450    0.000090
0                   64640    0.000314
29          Nerve Block 2    0.000425
19                  64493    0.001027
10                  62311    0.001376
12                  63661    0.001455
18                  64492    0.001488
13                  64494    0.001650
21                  27096    0.001683
11                  62310    0.001717
14                  64495    0.001782
17                  64491    0.001841
16                  64490    0.002239
15

C:\Users\James\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [243]:
#rfreg = grid.best_estimator
feature_cols =['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64494', '64495', '64633', '64490', '64491', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']##
rfreg2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=7, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
       

gridList =[]
for i in range(1,100):
    mean2escores = np.sqrt(-cross_val_score(rfreg2, X ,y, cv=5, scoring='mean_squared_error').mean())
    gridList += [mean2escores]
print np.array(gridList).mean()


#pandas.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_}).sort('importance')
#X_important = rfreg.transform(X, threshold=.01)
#rfreg.oob_score_

11.1258832782


In [38]:
"Grid search with cut down features"

print X_important


n_ests = range(1, 29, 2)
n_ests = range(1, 90, 2)
max_feats = range(1,len(X_important[0,])+1)
#max_feats = range(2,13)
param_grid = dict(n_estimators=n_ests,max_features =max_feats)


rfreg = RandomForestRegressor()

gridcut = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X_important,y)
print np.sqrt(-gridcut.best_score_)




NameError: name 'X_important' is not defined

In [37]:

print" best_features from grid search, cut features"
print gridcut.best_estimator_
print np.sqrt(-gridcut.best_score_)


 best_features from grid search, cut features


NameError: name 'gridcut' is not defined

In [137]:
"Run Random Forest Again"
max_feats = 7

#max_feats = range(2,13)
param_grid = dict(n_estimators=[41],max_features =[max_feats])
grid = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X,y)
print np.sqrt(-grid.best_score_)


NameError: name 'param_grid' is not defined

In [255]:
"My Transformer"


class EnsembleTranformer(TransformerMixin):
        def __init__(self, model, y):
            self.model = model
            self.y = y
        def transform(self,X,  **transform_params):
            if "RF" in str(self.model):
                X = X[alt_features]
                pass
            
            
            return np.array([self.model.predict(X)]).T
            
            
            
            #return X
           
# pipeline = feature union of EnsembleTranformer
        def fit(self, X, y=None, **fit_params):
            alt_features = ['63650', '64635', '64636', '63685', '63688', '63661', 'Lumbar Radiofrequency', 'Radiofrequency']#

            if "KNN" in str(self.model):
                
                X = X[alt_features]
                
                
            
            self.model.fit(X, y)
            return self

In [321]:
"LINEAR Ression Ensemble Sinan's"

"Dont Delete!"
"""from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
e = EnsembleTranformer(knn, response)
e.fit_transform(features)"""
"Dont Delete!"


knn = KNeighborsRegressor(n_neighbors=i)





rf_features = feature_cols

feature_cols =['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', 
'62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 
'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']

X =  dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']

#e =EnsembleTranformer(knn, y)
#print e.fit_transform(dfNotNull[knn_features]).shape

linreg =  LinearRegression() 

base = (2,11.5,11.5)
for k in range(1,25):
    print k
    linreg1 =  LinearRegression()
    linreg =  LinearRegression()
    knn = KNeighborsRegressor(n_neighbors=k)
    pipe = Pipeline([
    ('union', FeatureUnion([
        ('linreg',EnsembleTranformer(linreg, y)),
        ('knn', EnsembleTranformer(knn , y)),
         ])) 
    ,('mdl', linreg1 )
     ])

    mean2escores = np.sqrt(-cross_val_score(pipe, X, y, cv=5, scoring='mean_squared_error')).mean()
    meanabscores = -cross_val_score(pipe, X ,y, cv=5, scoring='mean_absolute_error').mean()
    print k
    print mean2escores
    print meanabscores
    base = (k, mean2escores,meanabscores )
        
        
        
print "Scores here*****"
print base






1
1
16.0234347479
9.38893197115
2
2
10.9672083015
7.05820083269
3
3
9.99039609551
6.65357904265
4
4
9.30558504102
6.24689221615
5
5
9.10835909524
6.21567542578
6
6
8.97927536448
6.14847712739
7
7
8.96654677066
6.13996306404
8
8
8.9299500805
6.11538550948
9
9
8.90623045232
6.11435316236
10
10
8.89468366889
6.11721091383
11
11
8.88071078079
6.10489587084
12
12
8.86910450335
6.09391306563
13
13
8.8744663204
6.10426725287
14
14
8.8592295435
6.09666717389
15
15
8.83421263865
6.09005263818
16
16
8.83716613818
6.08745056714
17
17
8.84742574881
6.08979179164
18
18
8.83126497654
6.08438150686
19
19
8.83315894539
6.09056238189
20
20
8.83353083286
6.08877644941
21
21
8.82502630903
6.08333832808
22
22
8.82012696377
6.08174753072
23
23
8.83076633668
6.09350185355
24
24
8.832911518
6.09124783999
Scores here*****
(24, 8.8329115180014224, 6.0912478399899541)


In [320]:
print "Ensemble with Average Lengths"

feature_cols =['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', 
'62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 
'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']

y = dfNotNull['TotalTimeMin']
X = dfNotNull[feature_cols]
numFolds = 5
kf = KFold(len(y), numFolds, shuffle=True)
sqrd =[]

for j in range(1,25):
    print j
    smlSqrd = []
    for train_indices, test_indices in kf:

                linreg =  LinearRegression()
                
                knn = KNeighborsRegressor(n_neighbors=9)
                features_train = X.T[train_indices];response_train = y[train_indices]
                features_test =  X.T[test_indices];response_test = y[test_indices]


                linreg.fit(features_train.T,response_train)
                predictionsLin = linreg.predict(features_test.T)

                KNNFeatures = ['63650', '64635', '64636', '63685', '63688', '63661', 'Lumbar Radiofrequency', 'Radiofrequency']
                rfreg.fit(features_train.T[KNNFeatures],response_train)
                predictionsK = rfreg.predict(features_test.T[KNNFeatures])
                predictions =(predictionsLin*1.2 + predictionsK*.8)/2
                mse =mean_squared_error(predictions, response_test)
                
                smlSqrd += [np.sqrt(mse)]
    
    sqrd += [np.array(smlSqrd).mean()]

print np.array(sqrd).mean()


Ensemble with Average Lengths
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
9.07180666787


In [282]:
print 55

55


In [156]:
"Predict on Out of Sample Data"

print predictions
pipe.fit
dfOOS = pandas.read_csv('OutOS.csv')
X = dfOOS[knn_features]
y = dfOOS['TotalTimeMin']
OutOfSamplePredictions = linreg.predict()








[ 23.68794299  96.72085827  29.53561915  28.87758398  22.23040777
  21.02400286  40.6248474   40.6248474   40.6248474   21.02400286
  42.48928747  41.8312523   22.23040777  42.48928747  36.10109435
  26.61548743  27.67117907  29.53561915  22.88844294  42.48928747
  42.48928747  42.48928747  29.53561915  28.87758398  42.48928747
  42.48928747  27.82189234  22.88844294  27.67117907  40.6248474
  96.72085827  21.02400286  29.53561915  22.88844294  42.48928747
  26.61548743  26.61548743  42.48928747  42.48928747  29.53561915
  40.6248474   40.6248474   21.02400286  27.67117907  42.48928747
  29.53561915  27.82189234  42.48928747  41.8312523   42.48928747
  22.88844294  28.47992751  27.82189234  22.23040777  34.23665427
  40.6248474   21.02400286  42.48928747  41.8312523   22.23040777
  98.58529835  42.48928747  29.53561915  41.8312523   42.48928747
  26.61548743  40.6248474   21.02400286  96.72085827  42.48928747
  22.88844294  41.8312523   28.47992751  98.58529835  41.8312523
  21.0240028

NameError: name 'knn_features' is not defined

In [328]:
print "Best Algorithms Best Scores!!!"

print """

Lingreg - no cpt codes deleted**********

[-141.8490452   -93.35778642  -61.82801688  -51.32990826  -55.10773072]
['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
mean squared error
8.80464997546
mean Abs Scores
6.09908443762

Lingreg - **********

['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r', 'Gender', 'F17', 'Age']
mean squared error
9.98563469287
mean Abs Scores
6.40693830594







RandomForest Mean of 100 Itearations ********************
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection','Medial Branch Block' ]##
rfreg2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=7, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

root mean squared error
10.2626357568




"get null scores************"
Total Average Null:
17.3521970207
11.1181800916

Procedure Average Null:
10.2979764338
6.56061055948


LINREG - Select KBest  *****************

['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', 'Radiofrequency', 'ESI', 'Spinal Cord Stimulator 2']

Root mean squared error:
10.3549488744
Mean Abs Scores:
6.58723506741


Ensemble - KNN and Linreg, ensembled by linreg *******************
KNN features = ['63650', '64635', '64636', '63685', '63688', '63661', 'Lumbar Radiofrequency', 'Radiofrequency']
LingerFeatures = ['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', 
'62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '3', '1r', 
'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']

Neighbors = 22
8.82012696377
6.08174753072







KNN
Pipeline(steps=[('anova', SelectKBest(k=8, score_func=<function f_classif at 0x000000000A934668>)), ('neigh', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=12, p=2,
          weights='uniform'))])
          
['63650', '64635', '64636', '63685', '63688', '63661', 'Lumbar Radiofrequency', 'Radiofrequency']
mean squared error
-85.3708057705
8.98504719777
mean Abs Scores
6.37585553713





"""


Best Algorithms Best Scores!!!


Lingreg - no cpt codes deleted**********

[-141.8490452   -93.35778642  -61.82801688  -51.32990826  -55.10773072]
['64640', '63650', '64634', '64480', '64635', '64636', '64484', '20605', '63685', '63688', '62311', '62310', '63661', '64633', '64492', '64493', '64450', '27096', 'Gender', 'Age', 'F17', '1r', 'ESI', 'Lumbar Radiofrequency', 'Nerve Block 2', 'Radiofrequency']
mean squared error
8.80464997546
mean Abs Scores
6.09908443762

Lingreg - **********

['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r', 'Gender', 'F17', 'Age']
mean squared error
9.98563469287
mean Abs Scores
6.40693830594







RandomForest Mean of 100 Itearations ********************
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection','Medial Branch Block' ]##
rfreg2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=2, max_leaf_nodes